In [32]:
import pandas as pd 
import numpy as np 

In [ ]:
def post_process(line):
    seq = ''.join(line.split()).upper().replace("-", "")
    seq = seq.replace('U','X').replace('Z', 'X').replace('O', 'X')
    return seq 

In [56]:
train = pd.read_csv('/home/zhuxh/data/TemStaPro/train.csv')

In [57]:
train.shape

(2175710, 9)

In [58]:
train['seq_len'] = train['sequence'].str.len()
train = train[train['seq_len'] < 1000].copy()
train['sequence'] = train['sequence'].apply(post_process)

In [60]:
train['seq_len'].quantile([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

0.1     98.0
0.2    142.0
0.3    183.0
0.4    226.0
0.5    268.0
0.6    312.0
0.7    363.0
0.8    429.0
0.9    535.0
Name: seq_len, dtype: float64

In [62]:
test = pd.read_csv('/home/zhuxh/data/TemStaPro/test.csv')

In [64]:
test.shape

(491100, 2)

In [65]:
test['seq_len'] = test['sequence'].str.len()
test = test[test['seq_len'] < 1000].copy()

In [66]:
test.shape 

(481487, 3)

In [67]:
test['sequence'] = test['sequence'].apply(post_process)

In [121]:
test.to_csv('/home/zhuxh/data/TemStaPro/test_kickup.csv', index=None)

In [102]:
from sklearn.model_selection import GroupKFold
group_kfold = GroupKFold(n_splits=5)
ar = list(group_kfold.split(train['sequence'], train['sequence'], train['cluster']))
dp = [train.index.values[i[1]] for i in ar]
for i,j in enumerate(dp):
    if i in [0, 1, 2, 3]:
        train.loc[j, 'set']='1train'
    else:
        train.loc[j, 'set']='2test'

In [120]:
train[train['set']=='1train'].to_csv('/home/zhuxh/data/TemStaPro/train_kickup.csv', index=None)
train[train['set']=='2test'].to_csv('/home/zhuxh/data/TemStaPro/val_kickup.csv', index=None)

In [2]:
#这里以bert模型为例，使用上述提到的函数
from transformers import BertTokenizer

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
text = "It's a nice day today!"

In [4]:
#tokenize,#仅用于分词
seg_words = tokenizer.tokenize(text)  
print("tokenize分词结果：\n",seg_words)

tokenize分词结果：
 ['it', "'", 's', 'a', 'nice', 'day', 'today', '!']


In [5]:
seg_word_id = tokenizer.convert_tokens_to_ids(seg_words)  
print("tokenize Id:\n",seg_word_id)

tokenize Id:
 [2009, 1005, 1055, 1037, 3835, 2154, 2651, 999]


In [6]:
#encode,进行分词和token转换，encode=tokenize+convert_tokens_to_ids
encode_text = tokenizer.encode(text)
print("encode结果：\n",encode_text)

encode结果：
 [101, 2009, 1005, 1055, 1037, 3835, 2154, 2651, 999, 102]


In [7]:
tokenizer.convert_ids_to_tokens(encode_text)

['[CLS]', 'it', "'", 's', 'a', 'nice', 'day', 'today', '!', '[SEP]']

In [8]:
text 

"It's a nice day today!"

In [21]:
#encode_plus,在encode的基础之上生成input_ids、token_type_ids、attention_mask
encode_plus_text = tokenizer.encode_plus(text, add_special_tokens=True, padding='max_length', max_length=20, return_tensors='pt')
print("encode_plus结果：\n",encode_plus_text)

encode_plus结果：
 {'input_ids': tensor([[ 101, 2009, 1005, 1055, 1037, 3835, 2154, 2651,  999,  102,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}


In [19]:
#encode_plus,在encode的基础之上生成input_ids、token_type_ids、attention_mask
encode_plus_text = tokenizer.batch_encode_plus(['it is good', 'ok'], add_special_tokens=True, padding='longest')
print("encode_plus结果：\n",encode_plus_text)

encode_plus结果：
 {'input_ids': [[101, 2009, 2003, 2204, 102], [101, 7929, 102, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 0, 0]]}
